# Le but de ce notebook est d'identifier des caractéristiques très simples permettant d'analyser des battements de coeur de foetus

In [ ]:
import os
import pandas as pd
import numpy as np

# permet de charger les fichiers matlab (*.mat)
from scipy.io import loadmat

# le répertoire de travail
directory = os.path.abspath('')

# répertoire où se trouvent toutes les données associés à ce challenge
data_directory = os.path.join(directory, 'Data')

# fichier CSV contenant les targets (1 / 0)
targets_path = os.path.join(data_directory, 'CTG_Challenge_files_GroundTruth.csv')

# répertoire où se trouvent les fichiers de données matlab
matlab_directory = os.path.join(data_directory, 'ctg_workshop_database')

# dans l'électrocardiogramme, nous avons 4 mesures par seconde
# chaque mesure correspondent à au nombre de battements de coeurs par minute
elements_en_1s = 4
elements_en_30minutes = int(elements_en_1s*30*60)
elements_en_1heure = int(elements_en_1s*3600)

# retourne tous les fichiers matlab présents dans le repertoire 'path'
def all_mat_files_in_directory(path: str):
    return [os.path.join(path,f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('.mat')]

# calcule la moyenne de la séquence ''fhr' en ignorant les NaN
def moyenne(fhr):
    return fhr[~np.isnan(fhr)].mean()

# calcule la std dev de la séquence ''fhr' en ignorant les NaN
def volatilite(fhr):
    return fhr[~np.isnan(fhr)].std()

# nombre d elements NaN dans la séquence 'fhr'
def nan_count(fhr):
    return np.count_nonzero(np.isnan(fhr))




# Chargement des données d'entraînement

In [ ]:
# chemin vers tous les fichiers matlab de la base d'entraînement
id_to_path = dict()
for filename in all_mat_files_in_directory(matlab_directory):
    filename_without_extension = os.path.splitext(os.path.basename(filename))[0]    
    id = int(filename_without_extension.lstrip('0'))
    id_to_path[id] = filename

# on charge les targets associés à chaque fichier d'entraînement
targets_df = pd.read_csv(targets_path)
id_to_target = dict()
for _, row in targets_df.iterrows():
    id_to_target[row['ChallengeID']] = row['TrueOutcome']

# lecture des battements de coeurs des foetus
id_to_fhr = dict()
all_lengths = []
for id, path in id_to_path.items():
    matlab_file = loadmat(path)
    id_to_fhr[id] = matlab_file['fhr'].flatten()

    

# Calcul de statistiques sur ces données d'entraînement
## (pour faciliter la recherche des caractéristiques)

In [ ]:

ids = []
targets = []

moyenne_full = []
volatilite_full = []
count_full = []
nan_count_full = []

moyenne_1ere_30minutes = []
volatilite_1ere_30minutes = []
count_1ere_30minutes = []
nan_count_1ere_30minutes = []

moyenne_derniere_30minutes = []
volatilite_derniere_30minutes = []
count_derniere_30minutes = []
nan_count_derniere_30minutes = []

moyenne_1ere_heure = []
volatilite_1ere_heure = []
count_1ere_heure = []
nan_count_1ere_heure = []

moyenne_derniere_heure = []
volatilite_derniere_heure = []
count_derniere_heure = []
nan_count_derniere_heure = []

for id in list(id_to_path.keys()):
    ids.append(id)
    targets.append(id_to_target[id])
    
    fhr_full = id_to_fhr[id]
    
    moyenne_full.append(moyenne(fhr_full))
    volatilite_full.append(volatilite(fhr_full)) 
    count_full.append(fhr_full.size)
    nan_count_full.append(nan_count(fhr_full))

    fhr_1ere_30minutes = fhr_full[:elements_en_30minutes]
    moyenne_1ere_30minutes.append(moyenne(fhr_1ere_30minutes))
    volatilite_1ere_30minutes.append(volatilite(fhr_1ere_30minutes)) 
    count_1ere_30minutes.append(fhr_1ere_30minutes.size)
    nan_count_1ere_30minutes.append(nan_count(fhr_1ere_30minutes))
    
    fhr_derniere_30minutes = fhr_full[-elements_en_30minutes:]
    moyenne_derniere_30minutes.append(moyenne(fhr_derniere_30minutes))
    volatilite_derniere_30minutes.append(volatilite(fhr_derniere_30minutes)) 
    count_derniere_30minutes.append(fhr_derniere_30minutes.size)
    nan_count_derniere_30minutes.append(nan_count(fhr_derniere_30minutes))
    
    fhr_1ere_heure = fhr_full[:elements_en_1heure]
    moyenne_1ere_heure.append(moyenne(fhr_1ere_heure))
    volatilite_1ere_heure.append(volatilite(fhr_1ere_heure)) 
    count_1ere_heure.append(fhr_1ere_heure.size)
    nan_count_1ere_heure.append(nan_count(fhr_1ere_heure))
    
    fhr_derniere_heure = fhr_full[-elements_en_1heure:]
    moyenne_derniere_heure.append(moyenne(fhr_derniere_heure))
    volatilite_derniere_heure.append(volatilite(fhr_derniere_heure)) 
    count_derniere_heure.append(fhr_derniere_heure.size)
    nan_count_derniere_heure.append(nan_count(fhr_derniere_heure))

    
# plus petite taille dans le dataset
# min([t[~np.isnan(t)].size for t in challengeid_to_fhr.values()])
 
# Sauvegarde de ces statistiques dans un DataFrame
fhr_stats = pd.DataFrame(
    {'ids': ids,
    'targets': targets,
    'moyenne_full' : moyenne_full,
    'volatilite_full' : volatilite_full,
    'count_full' : count_full,
    'nan_count_full' : nan_count_full,
    'moyenne_1ere_30minutes' : moyenne_1ere_30minutes,
    'volatilite_1ere_30minutes' : volatilite_1ere_30minutes,
    'count_1ere_30minutes' : count_1ere_30minutes,
    'nan_count_1ere_30minutes' : nan_count_1ere_30minutes,
    'moyenne_derniere_30minutes' : moyenne_derniere_30minutes,
    'volatilite_derniere_30minutes' : volatilite_derniere_30minutes,
    'count_derniere_30minutes' : count_derniere_30minutes,
    'nan_count_derniere_30minutes' : nan_count_derniere_30minutes,
    'moyenne_1ere_heure' : moyenne_1ere_heure,
    'volatilite_1ere_heure' : volatilite_1ere_heure,
    'count_1ere_heure' : count_1ere_heure,
    'nan_count_1ere_heure' : nan_count_1ere_heure,
    'moyenne_derniere_heure' : moyenne_derniere_heure,
    'volatilite_derniere_heure' : volatilite_derniere_heure,
    'count_derniere_heure' : count_derniere_heure,
    'nan_count_derniere_heure' : nan_count_derniere_heure,
    })

# on sauvegarde ces stats sur le disque
fhr_stats.to_csv(os.path.join(directory, 'fhr_stats.csv'), index=False)    

# Affichage d'un électrocardiogramme

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# id de l'électrocardiogramme à afficher
id = 61
# nous ne gardons que les 28 dernières minutes de l'enregistrement:
# (pour être aligné avec les fichiers pdf du répertoire 'ctg_data')
# Il y a 4 mesures par seconde: nous devons donc garder les 4*60*28 dernières valeurs
fhr = loadmat(id_to_path[id])['fhr'].ravel()[-1*4*60*28:]

# nombre d'éléments dans l'électrocardiogramme
n = len(fhr)

# Create an array of time points (assuming each heart rate measurement is taken at regular intervals)
time = np.arange(n)

# Plot the heart rate data
plt.figure(figsize=(20, 5))
plt.plot(time, fhr, linestyle='-', color='black', linewidth=1,label='Fetal Heart Rate')

# Add labels and title
plt.xlabel('Time')
plt.ylabel('Heart Rate (bpm)')
plt.title('Cardiogram of Heart Rates')
plt.legend()
plt.xlim(0, n-1)
plt.ylim(50, 210)
# Display grid
plt.grid(True)

# Show the plot
plt.show()

# Liste des caractéristiques

In [ ]:
import functools


id_to_data = id_to_fhr


@functools.lru_cache(maxsize=None)
def pourcentage_points_dans_intervalle(id, valeur_min: float, valeur_max: float) -> float:
    fhr = id_to_data[id]
    total_points = np.count_nonzero(~np.isnan(fhr))
    if total_points<=0:
        return 0
    total_points_dans_intervalle = np.count_nonzero((fhr > valeur_min) & (fhr < valeur_max) )
    return total_points_dans_intervalle/total_points

# calcul de la volatilité
@functools.lru_cache(maxsize=None)
def compute_id_volatility(id: int):
    return volatilite(id_to_data[id])
   
    
# calcul de l'etendue de la séquence associée à l'id 'id' en ignorant les NaN
@functools.lru_cache(maxsize=None)
def compute_id_range(id: int):
    data = id_to_data[id]
    return max(data)-min(data)
   
# calcul de la moyenne de la séquence associée à l'id 'id' en ignorant les NaN
@functools.lru_cache(maxsize=None)
def compute_id_mean(id: int):
    return moyenne(id_to_data[id])
       
# calcul de la moyenne sur l'ensemble du dataset
@functools.lru_cache(maxsize=None)
def global_mean() -> float :
    moyennes = []
    for id,data in id_to_data.items():
        moyennes.append(compute_id_mean(id)) 
    return np.mean(moyennes)
    
# 1ere caracteristique version 1A : sujet sain si le pourcentage de points dans un intervalle donné est supérieur à un seuil (avec ajustement à la moyenne).
def calcul_caracteristique1A(id: int, hyperparameters: dict) -> int:
    valeur_min_caracteristique1A = hyperparameters['min_value_fhr_caracteristique1A']
    valeur_min_caracteristique1A += compute_id_mean(id)-global_mean()
    valeur_max_caracteristique1A = valeur_min_caracteristique1A +hyperparameters['range_value_fhr_caracteristique1A']
    pourcentage_points_dans_intervalle_caracteristique1A = pourcentage_points_dans_intervalle(id, valeur_min_caracteristique1A, valeur_max_caracteristique1A)
    return 0 if (pourcentage_points_dans_intervalle_caracteristique1A > hyperparameters['seuil_caracteristique1A']) else 1

# 1ere caracteristique version 1B : sujet sain si le pourcentage de points dans un intervalle donné est supérieur à un seuil (sans ajustement à la moyenne).
def calcul_caracteristique1B(id: int, hyperparameters: dict) -> int:
    valeur_min_caracteristique1B = hyperparameters['min_value_fhr_caracteristique1B']
    valeur_max_caracteristique1B = valeur_min_caracteristique1B +hyperparameters['range_value_fhr_caracteristique1B']
    pourcentage_points_dans_intervalle_caracteristique1B = pourcentage_points_dans_intervalle(id, valeur_min_caracteristique1B, valeur_max_caracteristique1B)
    return 0 if (pourcentage_points_dans_intervalle_caracteristique1B > hyperparameters['seuil_caracteristique1B']) else 1

# 2eme caracteristique version 2A : sujet sain si le pourcentage de points dans un intervalle donné est inférieur à un seuil (avec ajustement à la moyenne).
def calcul_caracteristique2A(id: int, hyperparameters: dict) -> int:
    valeur_min_caracteristique2A = hyperparameters['min_value_fhr_caracteristique2A']
    valeur_min_caracteristique2A += compute_id_mean(id)-global_mean()
    valeur_max_caracteristique2A = valeur_min_caracteristique2A +hyperparameters['range_value_fhr_caracteristique2A']
    pourcentage_points_dans_intervalle_caracteristique2A = pourcentage_points_dans_intervalle(id, valeur_min_caracteristique2A, valeur_max_caracteristique2A)
    return 0 if (pourcentage_points_dans_intervalle_caracteristique2A < hyperparameters['seuil_caracteristique2A']) else 1

# 2eme caracteristique version 2B : sujet sain si le pourcentage de points dans un intervalle donné est inférieur à un seuil (sans ajustement à la moyenne).
def calcul_caracteristique2B(id: int, hyperparameters: dict) -> int:
    valeur_min_caracteristique2B = hyperparameters['min_value_fhr_caracteristique2B']
    valeur_max_caracteristique2B = valeur_min_caracteristique2B +hyperparameters['range_value_fhr_caracteristique2B']
    pourcentage_points_dans_intervalle_caracteristique2B = pourcentage_points_dans_intervalle(id, valeur_min_caracteristique2B, valeur_max_caracteristique2B)
    return 0 if (pourcentage_points_dans_intervalle_caracteristique2B < hyperparameters['seuil_caracteristique2B']) else 1

# 3eme caracteristique: sujet sain si l'étendue de l'électrocardiogramme est inférieur à un seuil.
def calcul_caracteristique3(id: int, hyperparameters: dict) -> int:
    etendue_caracteristique3 = compute_id_range(id)
    return 0 if (etendue_caracteristique3 < hyperparameters['seuil_caracteristique3']) else 1

# 4eme caracteristique: sujet sain si le pourcentage de points autour de la moyenne est supérieur à un seuil.
def calcul_caracteristique4(id: int, hyperparameters: dict) -> int:
    range_caracteristique4 = hyperparameters['range_caracteristique4']
    moyenne = compute_id_mean(id)
    pourcentage_points_dans_intervalle_caracteristique4 = pourcentage_points_dans_intervalle(id, moyenne-range_caracteristique4, moyenne+range_caracteristique4)
    return 0 if (pourcentage_points_dans_intervalle_caracteristique4 > hyperparameters['seuil_caracteristique4']) else 1

# 5eme caracteristique: sujet sain si la moyenne de l'électrocardiogramme est inférieur à un seuil.
def calcul_caracteristique5(id: int, hyperparameters: dict) -> int:
    valeur_min_caracteristique5 = hyperparameters['min_value_caracteristique5']
    seuil_caracteristique5 = hyperparameters['seuil_caracteristique5']
    valeur_max_caracteristique5 = valeur_min_caracteristique5 +seuil_caracteristique5
    moyenne_caracteristique5 = compute_id_mean(id)
    return 0 if ((moyenne_caracteristique5>=valeur_min_caracteristique5) and (moyenne_caracteristique5<=valeur_max_caracteristique5)) else 1

# 6eme caracteristique: sujet sain si le % de points au dessus de la moyenne de l'électrocardiogramme est inférieur à un seuil.
def calcul_caracteristique6(id: int, hyperparameters: dict) -> int:
    pourcentage_caracteristique6 = hyperparameters['pourcentage_caracteristique6']
    moyenne = compute_id_mean(id)
    pourcentage_points_au_dessus_de_la_moyenne_caracteristique6 = pourcentage_points_dans_intervalle(id, moyenne, moyenne+9999)
    return 0 if (pourcentage_points_au_dessus_de_la_moyenne_caracteristique6 < hyperparameters['pourcentage_caracteristique6']) else 1

# 7eme caracteristique: sujet sain si la volatilité de l'électrocardiogramme est inférieur à un seuil.
def calcul_caracteristique7(id: int, hyperparameters: dict) -> int:
    volatilite_caracteristique7 = compute_id_volatility(id)
    return 0 if (volatilite_caracteristique7 < hyperparameters['seuil_caracteristique7']) else 1

# 8eme caracteristique: sujet sain si le pourcentage de points dans l'intervalle [ k1 * moyenne, (1+k2) * moyenne] est supérieur à un seuil.
def calcul_caracteristique8(id: int, hyperparameters: dict) -> int:
    k1_caracteristique8 = hyperparameters['k1_caracteristique8']
    k2_caracteristique8 = hyperparameters['k2_caracteristique8']
    moyenne = compute_id_mean(id)
    pourcentage_points_dans_intervalle_caracteristique8 = pourcentage_points_dans_intervalle(id, k1_caracteristique8*moyenne, (1+k2_caracteristique8)*moyenne)
    return 0 if (pourcentage_points_dans_intervalle_caracteristique8 > hyperparameters['seuil_caracteristique8']) else 1

# 9eme caracteristique: sujet sain si le pourcentage de points dans l'intervalle [ 0, k] est inférieur à un seuil.
def calcul_caracteristique9(id: int, hyperparameters: dict) -> int:
    k_caracteristique9 = hyperparameters['k_caracteristique9']
    pourcentage_points_dans_intervalle_caracteristique9 = pourcentage_points_dans_intervalle(id, 0, k_caracteristique9)
    return 0 if pourcentage_points_dans_intervalle_caracteristique9 < hyperparameters['seuil_caracteristique9'] else 1

# 10eme caracteristique: sujet sain si le pourcentage de points dans l'intervalle [ 0, k] est supérieur à un seuil.
def calcul_caracteristique10(id: int, hyperparameters: dict) -> int:
    k_caracteristique10 = hyperparameters['k_caracteristique10']
    pourcentage_points_dans_intervalle_caracteristique10 = pourcentage_points_dans_intervalle(id, 0, k_caracteristique10)
    return 0 if pourcentage_points_dans_intervalle_caracteristique10 > hyperparameters['seuil_caracteristique10'] else 1
   
    

# Hyperparameters Management

In [ ]:
#pip install hyperopt

import numpy as np
from sklearn.metrics import f1_score
from typing import List
import math
from hyperopt import fmin, tpe, space_eval, hp, Trials, rand as hyperopt_rand
import hashlib
import random
import time
import sys


current_best = 0.5
stats_hpo = dict()
last_time_display_stats_hpo = time.time()
   
        
def compute_f1_score(TP: int, TN: int, FP: int, FN: int):
    return (2*TP)/(1.0*TP+FP+FN)
        
def compute_accuracy_target0(TP: int, TN: int, FP: int, FN: int):
    return TN/(1.0*TN+FP)

def compute_accuracy_target1(TP: int, TN: int, FP: int, FN: int):
    return TP/(1.0*TP+FN)

def compute_score(TP: int, TN: int, FP: int, FN: int):
    return compute_average_accuracy(TP,TN,FP,FN)
    #return compute_f1_score(TP,TN,FP,FN)
    #return min(compute_accuracy_target0(TP,TN,FP,FN),compute_accuracy_target1(TP,TN,FP,FN))

def compute_average_accuracy(TP: int, TN: int, FP: int, FN: int):
    return (compute_accuracy_target0(TP,TN,FP,FN)+compute_accuracy_target1(TP,TN,FP,FN))/2
        
def compute_matrice_de_confusion(id_to_predictions, id_to_target) -> (int, int, int,int):
        TP = 0
        TN = 0
        FP = 0
        FN = 0
        for id, data in id_to_predictions.items():
            target = id_to_target[id]
            prediction = id_to_predictions[id]
            if prediction == target: # bonne prediction
                if target == 1:
                    TP += 1
                else:
                    TN += 1
            else: # erreur dans la prediciton
                if prediction == 1:
                    FP += 1
                else:
                    FN += 1
        return (TP,TN,FP,FN)

        


def compute_single_prediction(id: str, hyperparameters: dict) -> int:  
    valeurs_caracteristiques_a_utiliser = []
    for c in hyperparameters['caracteristiques_a_utiliser'].split('+'):
        # 1ere caracteristique version 1A(avec ajustement à la moyenne)
        if c == '1A':
            valeurs_caracteristiques_a_utiliser.append(calcul_caracteristique1A(id, hyperparameters))
        elif c == '1B':
            valeurs_caracteristiques_a_utiliser.append(calcul_caracteristique1B(id, hyperparameters))
        elif c == '2A':
            valeurs_caracteristiques_a_utiliser.append(calcul_caracteristique2A(id, hyperparameters))
        elif c == '2B':
            valeurs_caracteristiques_a_utiliser.append(calcul_caracteristique2B(id, hyperparameters))
        elif c == '3':
            valeurs_caracteristiques_a_utiliser.append(calcul_caracteristique3(id, hyperparameters))
        elif c == '4':
            valeurs_caracteristiques_a_utiliser.append(calcul_caracteristique4(id, hyperparameters))
        elif c == '5':
            valeurs_caracteristiques_a_utiliser.append(calcul_caracteristique5(id, hyperparameters))
        elif c == '6':
            valeurs_caracteristiques_a_utiliser.append(calcul_caracteristique6(id, hyperparameters))
        elif c == '7':
            valeurs_caracteristiques_a_utiliser.append(calcul_caracteristique7(id, hyperparameters))
        elif c == '8':
            valeurs_caracteristiques_a_utiliser.append(calcul_caracteristique8(id, hyperparameters))
        elif c == '9':
            valeurs_caracteristiques_a_utiliser.append(calcul_caracteristique9(id, hyperparameters))
        elif c == '10':
            valeurs_caracteristiques_a_utiliser.append(calcul_caracteristique10(id, hyperparameters))
        else:
            raise Exception(f'caracteristique invalide {c}')
    if not all_elements_same(valeurs_caracteristiques_a_utiliser):
        return hyperparameters['label_si_resultats_differents']
    return valeurs_caracteristiques_a_utiliser[0]
    
    
def all_elements_same(data):
    first_element = data[0]
    return all(element == first_element for element in data)


def compute_toutes_les_predictions(hyperparameters: dict) -> dict:
    id_to_predictions = dict()
    for id in id_to_data.keys():
        id_to_predictions[id] = compute_single_prediction(id, hyperparameters)
    return id_to_predictions
    
    
    
def train(hyperparameters: dict) -> dict:
    id_to_predictions = compute_toutes_les_predictions(hyperparameters)
    (TP,TN,FP,FN) = compute_matrice_de_confusion(id_to_predictions, id_to_target)
    metrics = dict()
    metrics['TP'] = TP
    metrics['TN'] = TN
    metrics['FP'] = FP
    metrics['FN'] = FN
    metrics['accuracy_target0'] = compute_accuracy_target0(TP,TN,FP,FN)
    metrics['accuracy_target1'] = compute_accuracy_target1(TP,TN,FP,FN)
    metrics['f1_score'] = compute_f1_score(TP,TN,FP,FN)
    metrics['average_accuracy'] = compute_average_accuracy(TP,TN,FP,FN)
    current_score = compute_score(TP,TN,FP,FN)
    metrics['score'] = current_score
    update_stats_hpo(current_score, hyperparameters)
    global last_time_display_stats_hpo
    if (time.time()-last_time_display_stats_hpo)>600:
        save_stats_hpo(False)
        last_time_display_stats_hpo = time.time()
    global current_best
    if not current_best or current_score>current_best:
        current_best = current_score
        print(f"new best score {round(current_best,4)} for hyperparameters {[c for c in hyperparameters.items() if c[1] is not None]}")
        save_model(hyperparameters, metrics)
        save_stats_hpo(False)
    return metrics

    
def update_stats_hpo(score:float, hyperparameters: dict) -> None:
    for hpo_key, hpo_value in hyperparameters.items():
        if hpo_value is None:
            continue
        if hpo_key not in stats_hpo:
            stats_hpo[hpo_key] = dict()
        if hpo_value not in stats_hpo[hpo_key]:
            stats_hpo[hpo_key][hpo_value] = [0,0]
        stats_hpo[hpo_key][hpo_value][0] += 1
        stats_hpo[hpo_key][hpo_value][1] += score

    
# the objective used for Hyperparameters Optimization (HPO)
# it is the function to minimize
def objective(sample_from_search_space):
    hyperparameters = fix_hyperparameters(sample_from_search_space)
    model_name = get_model_name(hyperparameters)
    if model_name in already_processed_model_names_with_hpo:
        metrics = already_processed_model_names_with_hpo[model_name]
    else:
        metrics = train(hyperparameters)
        already_processed_model_names_with_hpo[model_name] = metrics
    # we want to minimize this objective, so we return -score
    return -metrics['score']

hpo_session_id = str(int(100*time.time()))

def save_stats_hpo(display: bool):
    res = stats_hpo_to_str()
    try:
        path = os.path.join(directory, 'hpo_'+hpo_session_id+".txt")
        with open(path, 'w') as f:
            f.write(res)
    except Exception as e:
        print(f'failed to save hp')
    if display:
        print(res)

# path to the last model saved
last_path_for_save_model = None
        
def save_model(hyperparameters: dict, metrics: dict) -> None:
    global last_path_for_save_model
    try:
        score = metrics['score']
        path = os.path.join(directory, 'hpo_'+hpo_session_id+'_'+hyperparameters['caracteristiques_a_utiliser']+'_'+str(score)+".txt")
        with open(path, 'w') as f:
            text = hyperparameters_to_str(hyperparameters)
            for m, v in metrics.items():
                text += f'\n#{m}={v}'
            f.write(text)
        try:
            if last_path_for_save_model and os.path.isfile(last_path_for_save_model):
                os.remove(last_path_for_save_model)
        except Exception as e:
            print(f'failed to delete file {last_path_for_save_model}: {e}')
        last_path_for_save_model = path
    except Exception as e:
        print(f'failed to save hp: {e}')
    
def stats_hpo_to_str() -> str:
    max_intervals = 5
    result = ""
    for hpo_key, hpo_key_stats in stats_hpo.items():
        result += f"stats for key '{hpo_key}':\n"
        all_values = list(hpo_key_stats.keys())
        old_value_to_new_value = dict()
        if len(all_values)>max_intervals and (isinstance(all_values[0], int) or isinstance(all_values[0], float)):
            min_value = float(min(all_values))
            max_value = float(max(all_values))
            for v in all_values:
                index_interval = int (max_intervals * (float(v-min_value)/(max_value-min_value)))
                index_interval = min(index_interval, max_intervals-1)
                min_value_interval = min_value+index_interval* (max_value-min_value)/max_intervals
                max_value_interval = min_value_interval+ (max_value-min_value)/max_intervals
                new_key = f'[{round(min_value_interval,2)}, {round(max_value_interval,2)}]'
                if new_key not in old_value_to_new_value:
                    old_value_to_new_value[new_key] = [0,0]
                old_value_to_new_value[new_key][0] += hpo_key_stats[v][0]
                old_value_to_new_value[new_key][1] += hpo_key_stats[v][1]
        else:
            for v in all_values:
                old_value_to_new_value[v] = hpo_key_stats[v]     
        
        for value, value_stats in sorted(old_value_to_new_value.items(), key=lambda item: item[1][1]/item[1][0], reverse=True):
            count = value_stats[0]
            avg_score = value_stats[1]/count
            result += f'\t{value} : {avg_score} ({count} samples)\n'
    return result
        
def extract_caracteristique_id(key: str):
    token ="_caracteristique"
    idx = key.find(token)
    if idx<0:
        return None
    return key[idx+len(token):]

# When conducting an HPO search, some hyperparameters may exhibit inconsistent values.
# This method aims to address those inconsistencies.
def fix_hyperparameters(hyperparameters: dict) -> dict :
    res = dict(hyperparameters)
    for key in list(res.keys()):
        caracteristique_id = extract_caracteristique_id(key)
        if caracteristique_id and caracteristique_id not in res['caracteristiques_a_utiliser'].split('+'):
            del res[key]
    if '+' not in res['caracteristiques_a_utiliser']:
        res['label_si_resultats_differents'] = None
    return res


# Transform the dictionary of hyperparameters 'hyperparameters' into string.
def hyperparameters_to_str(hyperparameters: dict) -> str:
    sorted_hyperparameters = sorted (hyperparameters.items())
    return "\n".join([hyperparameter_name+" = "+str(hyperparameter_value) for (hyperparameter_name,hyperparameter_value) in sorted_hyperparameters if hyperparameter_value is not None])

def get_model_name(hyperparameters: dict) -> str:
    file_content = hyperparameters_to_str(hyperparameters)
    return compute_hash(file_content, 10)

def compute_hash(input_string, max_length):
    # Calculate MD5 hash of the input string
    md5_hash = hashlib.md5(input_string.encode('ascii')).hexdigest().upper()
    # Return the hash truncated to the max_length
    return md5_hash[:max_length]

def launch_hpo_for_transformer_model(max_evals: int):
    seed = random.randint(0, 100000)
    print(f'using seed: {seed}')
    rstate = np.random.default_rng(seed)
    best_indexes = fmin(
        fn=objective,  # "Loss" function to minimize
        space=search_space,  # Hyperparameter space
        #algo=hyperopt_rand.suggest, #Random Search
        algo=tpe.suggest,  # Tree-structured Parzen Estimator (TPE)
        max_evals=max_evals,  # Perform 'max_evals' trials
        max_queue_len = 10,
        rstate =rstate,
    )

    # Get the best parameters
    best  = space_eval(search_space, best_indexes)
    print(f"Found minimum after {max_evals} trials:")
    print([c for c in best.items() if c[1] is not None])
    return best


search_space = {

    # 1ere caracteristique version 1A : sujet sain si le pourcentage de points dans un intervalle donné est supérieur à un seuil (avec ajustement à la moyenne).
    'min_value_fhr_caracteristique1A': hp.choice('min_value_fhr_caracteristique1A', list(range(50,150+1,1))), # 95 104
    'range_value_fhr_caracteristique1A': hp.choice('range_value_fhr_caracteristique1A', list(range(30,150+1,1))), # 125 , 115 , 85 68
    'seuil_caracteristique1A': hp.choice('seuil_caracteristique1A', [(i * 0.001) for i in range(800, 1000+1,1)]), # 0.95 0.96 0.91

    # 1ere caracteristique version 1B : sujet sain si le pourcentage de points dans un intervalle donné est supérieur à un seuil (sans ajustement à la moyenne).
    'min_value_fhr_caracteristique1B': hp.choice('min_value_fhr_caracteristique1B', list(range(50,150+1,1))), # 93
    'range_value_fhr_caracteristique1B': hp.choice('range_value_fhr_caracteristique1B', list(range(30,150+1,1))), # 72
    'seuil_caracteristique1B': hp.choice('seuil_caracteristique1B', [(i * 0.001) for i in range(800, 1000+1,1)]), # 0.906

    # 2eme caracteristique version 2A : sujet sain si le pourcentage de points dans un intervalle donné est inférieur à un seuil (avec ajustement à la moyenne).
    'min_value_fhr_caracteristique2A': hp.choice('min_value_fhr_caracteristique2A', list(range(15,50+1,1))),  # 35 , 20, 38 28
    'range_value_fhr_caracteristique2A': hp.choice('range_value_fhr_caracteristique2A', list(range(30,100+1,1))), # 65 , 54 64
    'seuil_caracteristique2A': hp.choice('seuil_caracteristique2A', [(i * 0.001) for i in range(0, 100+1,1)]), # 0.04 , 0.03 0.01  003

    # 2eme caracteristique version 2B : sujet sain si le pourcentage de points dans un intervalle donné est inférieur à un seuil (sans ajustement à la moyenne).
    'min_value_fhr_caracteristique2B': hp.choice('min_value_fhr_caracteristique2B', list(range(10,100+1,2))), # 14
    'range_value_fhr_caracteristique2B': hp.choice('range_value_fhr_caracteristique2B', list(range(20,120+1,5))), # 70
    'seuil_caracteristique2B': hp.choice('seuil_caracteristique2B', [(i * 0.001) for i in range(0, 100+1,1)]), # 0.03

    # 3eme caracteristique: sujet sain si l'étendue de l'électrocardiogramme est inférieur à un seuil.
    'seuil_caracteristique3': hp.choice('seuil_caracteristique3', list(range(100,150+1,1))), # 124

    # 4eme caracteristique: sujet sain si le pourcentage de points autour de la moyenne est supérieur à un seuil.
    'range_caracteristique4': hp.choice('range_caracteristique4', list(range(20,75+1,1))), # 43
    'seuil_caracteristique4': hp.choice('seuil_caracteristique4', [(i * 0.001) for i in range(900, 1000+1,1)]), # 0.96

    # 5eme caracteristique: sujet sain si la moyenne de l'électrocardiogramme est dans l'intervalle [k1, k2]
    'min_value_caracteristique5': hp.choice('min_value_caracteristique5', list(range(122-40,122+40+1,1))),
    'seuil_caracteristique5': hp.choice('seuil_caracteristique5', list(range(9-6,9+6+1,1))),

    # 6eme caracteristique: sujet sain si le % de points au dessus de la moyenne de l'électrocardiogramme est inférieur à un seuil.
    'pourcentage_caracteristique6': hp.choice('pourcentage_caracteristique6', [(i * 0.01) for i in range(61-30, 61+30+1,1)]),

    # 7eme caracteristique: sujet sain si la volatilité de l'électrocardiogramme est inférieur à un seuil.
    'seuil_caracteristique7': hp.choice('seuil_caracteristique7', [(i * 0.01) for i in range(1880, 1980+1,1)]),

    # 8eme caracteristique: sujet sain si le pourcentage de points dans l'intervalle [ k1 * moyenne, (1+k2) * moyenne] est supérieur à un seuil.
    'k1_caracteristique8': hp.choice('k1_caracteristique8', [(i * 0.01) for i in range(0, 100+1,1)]), # 0.26 0.45 0.72  0.92 0.44
    'k2_caracteristique8': hp.choice('k2_caracteristique8', [(i * 0.01) for i in range(0, 50+1,1)]),  # 0.13 0.33 0.1
    'seuil_caracteristique8': hp.choice('seuil_caracteristique8', [(i * 0.01) for i in range(70, 100+1,1)]), #0.87 0.96

    # 9eme caracteristique: sujet sain si le pourcentage de points dans l'intervalle [ 0, k] est inférieur à un seuil.
    'k_caracteristique9': hp.choice('k_caracteristique9', list(range(0, 150+1,1))), # 94
    'seuil_caracteristique9': hp.choice('seuil_caracteristique9', [(i * 0.001) for i in range(0, 200+1,1)]), # 0.03

    # 10eme caracteristique: sujet sain si le pourcentage de points dans l'intervalle [ 0, k] est supérieur à un seuil.
    'k_caracteristique10': hp.choice('k_caracteristique10', list(range(100,250+1,1))), # 156
    'seuil_caracteristique10': hp.choice('seuil_caracteristique10', [(i * 0.001) for i in range(800, 1000+1,1)]), # 0.94

    'caracteristiques_a_utiliser': hp.choice('caracteristiques_a_utiliser', ['1A+7']),
    'label_si_resultats_differents': hp.choice('label_si_resultats_differents', [0, 1]),
}


if len(sys.argv) >=2 and str.isdigit(sys.argv[1][0]):
    caracteristiques_a_utiliser = sys.argv[1].split(',')
    print(f'caracteristiques_a_utiliser will be set to {caracteristiques_a_utiliser}')
    search_space['caracteristiques_a_utiliser'] =  hp.choice('caracteristiques_a_utiliser', caracteristiques_a_utiliser)


max_evals = 100
if len(sys.argv) >=3 and str.isdigit(sys.argv[2]):
    print(f'max_evals will be set to {sys.argv[2]}')
    max_evals = int(sys.argv[2])
print(f'max_evals value is {max_evals}')

# Uncomment following line to enable HPO
already_processed_model_names_with_hpo = dict()
start_time = time.time()
best = launch_hpo_for_transformer_model(max_evals)
print(f'hpo took {round(time.time()-start_time,2)}s')
save_stats_hpo(True)
